# feat_treat() class

### Example of feat_treat()

In [30]:
import pandas as pd
# import data which was subset using R
df_y6= pd.read_csv('/home/cristianromero/Documents/Datasets/kantar_6_targets.csv')
df_x6= pd.read_csv('/home/cristianromero/Documents/Datasets/kantar_6_features.csv')

# remove extra id columns (relic of exporting data from R)
df_y6=df_y6.iloc[:,1]
df_x6=df_x6.iloc[:,1:len(df_x6)]

In [1]:
treatment1 = feat_treat(df_x6,df_y6,42)

In [2]:
# impute missing values
treatment1.check_na(50)

In [5]:
# return identified columns
treatment1.na_col

In [35]:
# remove 
treatment1.handle_na('remove',treatment1.na_col)

In [6]:
treatment1.check_na() # run with no argument to return all columns with missing values
treatment1.handle_na('median',treatment1.na_col)

In [7]:
treatment1.check_na()

In [48]:
treatment1.encode()

In [31]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier

In [8]:
treatment1.sample_tune_test(tuning_iter=50,model=classifer)

In [ ]:
treatment1.metrics

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier
# impute
treatmeant1=feat_treat(df_x6,df_y6,42)
treatmeant1.check_na(50)
treatmeant1.handle_na('remove',treatmeant1.na_col)
treatmeant1.check_na()
treatmeant1.handle_na('median',treatmeant1.na_col)
# test
treatmeant1.encode()
treatment1.sample_tune_test(tuning_iter=100,model=classifer)
treatmeant1.metrics

In [13]:
# Feature treatment class
# Use to optimize feature selection, sampling, and hyperparameter tuning for model testing
# Stream-lines testing process and brings together packages like scikit-learn, skopt, pandas, numpy 
# create instances of feature treatments 
# returns performance metrics for treatments with visual

# Run Time: 
# RandomForestClassifier = 4.5 iterations/min
# XGBClassifier = 5.2 iterations/min

# required arguments: 
# X = feature pandas dataframe long format
# y = dependent variable pandas dataframe long format
# random_state = seed for pseudo random number generator to be used throughout treatments

import pandas as pd
import numpy as np

class feat_treat:
    def __init__(self,X,y,random_state):
        self.X=X
        self.y=y
        self.random_state=random_state
        self.encoded=False
        self.na_col='update with check_na(percent_threshold)'
        self.metrics='No metrics available'
        
        # Low variance filter 
        # check variance of each column
        # remove column if variance is less than var_threshold
        drop=[]
        var_threshold = 0.004975
        for col in self.X.columns:
            if self.X[col].dtype!='object': 
                if np.var(self.X[col]) < var_threshold:
                    drop.append(col)
        self.X=self.X.drop(columns=drop)
        
        # GET TO KNOW YOUR DATA SET
        # return dataframe indices for categorical datatypes
        cat_col=[]
        for i in self.X.columns:
            if self.X[i].dtype == 'object':
                cat_col.append(self.X.columns.get_loc(i))
        print("categorical column indices: ",cat_col,"\n ")
    
        # return base rate
        unique_elements, counts_elements = np.unique(y, return_counts=True)
        DV_classes_df = pd.DataFrame({'Class': unique_elements,
                                      'Count': counts_elements})
        print(DV_classes_df.to_string(index=False),"\n ")
        print("Base rate: ",counts_elements[1]/sum(counts_elements))
    
    

    def check_na(self,percent_threshold=0):
        # check for missing values  
        percent_missing = self.X.isnull().sum() * 100 / len(self.X)
        column_name = [percent_missing.index[i] for i in range(0,len(self.X.columns)) if percent_missing[i]>0]
        percent_na = [percent_missing[i] for i in range(0,len(self.X.columns)) if percent_missing[i]>0]
        missing_value_df = pd.DataFrame({'column_name': column_name,
                                         'percent_na': percent_na})
        missing_value_df.sort_values('percent_na', inplace=True)
        print(missing_value_df, "\n ")
        self.na_col=column_name
    
    
    
    def handle_na(self,strategy,columns):
        # handle missing values
        # methods: mean, median, mode, value, remove
        from scipy import stats
        
        if isinstance(strategy,int)==True or isinstance(strategy,float)==True:
            for i in columns:
                self.X[i]=self.X[i].fillna(strategy)
        else:
            strategy1=''.join(strategy.split()).lower()
            if strategy1=='mean':
                for i in columns:
                    mean=self.X[i].mean()
                    self.X[i].fillna(mean,inplace=True)
            elif strategy1=='median':
                for i in columns:
                    median=self.X[i].median()
                    self.X[i].fillna(median,inplace=True)                            
            elif strategy1=='mode':
                for i in columns:
                    mode = self.X[i].mode()[0]
                    self.X[i].fillna(mode,inplace=True)                      
            elif strategy1=='remove':
                self.X.drop(columns=self.na_col,inplace=True) 
            else:
                for i in columns:
                    self.X[i].fillna(strategy,inplace=True)                 
         
    
    
    def pcc_filter(self,k):
        if isinstance(k,int)==True or isinstance(k,float)==True:
            # check pearson coefficient for linear correlation between DV and each feature 
            # remove column if PCC is less than correlation limit
  
            df=pd.concat([self.y, self.X], axis=1, ignore_index=True)
#           create pearson correlation coefficient matrix
            cor = df.corr()
            corr_limit=k
            cor_target = abs(cor.iloc[:,0])
            relevant_features = cor_target[cor_target>corr_limit]
            relevant_features=relevant_features.iloc[1:]
            rel_col =[]
            for i in relevant_features.index:
                rel_col.append(i-1) 
#           drop all columns except relevant ones
            self.X=self.X.iloc[:,rel_col]
#           check which columns are kept 
            keep_col=self.X.columns
            print(keep_col)
            print(len(keep_col))            
#       section to include pcc optimization loop
        else:
            k = ''.join(k.split()).lower()
            k = k[:3]
            if k == 'opt':
                pass

    
    
    def collinear(self,k):
#       check pearson coefficient for linear correlation between features (collinearity)
#       remove if PCC is greater than collinear limit
#       recreate pearson correlation matrix
        cor = self.X.corr()
        feat_targets=[]
        feat_remove=[]
        ignore=[]
#       set correlation limit
        colinear_corr_limit=k
#       check for collinearity
        for i in self.X.columns:
            cor_target = abs(cor.loc[:,i])
            feat_targets.append((i,cor_target[cor_target>colinear_corr_limit]))  
        # remove collinear features
        for i in feat_targets:
            ignore.append(i[0])
            for j in i[1].index:
                if j not in ignore:
                    feat_remove.append(j)

        feat_remove=set(feat_remove)
        self.X=self.X.drop(columns=feat_remove)
        # check which columns are kept 
        keep_col=self.X.columns
        print("columns remaining: ",keep_col)
        print(len(keep_col), " columns")
                      
    
    def encode(self, strategy=None):
        if(strategy=='dummy'):
            self.X=pd.get_dummies(self.X)
            self.encoded=True    
        else:
            from sklearn import preprocessing
            cat_col=[]
            for i in self.X.columns:
                if self.X[i].dtype == 'object':
                    cat_col.append(i)
            le = preprocessing.LabelEncoder()
            for col in cat_col:
                self.X[col]=le.fit_transform(self.X[col])            
            self.encoded=True    
        
    
    
    def rfe(self,n,rfe_model=None):
        if isinstance(n,int)==True or isinstance(n,float)==True:
            if rfe_model==None:
                from sklearn.linear_model import LogisticRegression
                rfe_model=LogisticRegression(solver='lbfgs')
                from sklearn.feature_selection import RFE
                selector = RFE(estimator=rfe_model, n_features_to_select = n).fit(self.X,self.y)
                keep = [i for i in range(0,len(selector.support_)) if selector.support_[i]==True]
                self.X=self.X.iloc[:,keep]
            else:
                from sklearn.feature_selection import RFE
                selector = RFE(estimator=rfe_model, n_features_to_select = n).fit(self.X,self.y)
                keep = [i for i in range(0,len(selector.support_)) if selector.support_[i]==True]
                self.X=self.iloc[:,keep]
#       section to include optimization loop
        else:
            n = ''.join(n.split()).lower()
            n = n[:3]
            if n == 'opt':
                pass                

            

    def skb(self,k,score_func=None):
        if self.encoded == True:
            if isinstance(k,int)==True or isinstance(k,float)==True:
                if score_func==None:
                    from sklearn.feature_selection import SelectKBest, chi2
                    skb =SelectKBest(score_func=chi2, k=k).fit(self.X,self.y)
                    self.X = self.X.iloc[:,skb.get_support(indices=True)]
                else:
                    from sklearn.feature_selection import SelectKBest, score_func
                    skb = SelectKBest(score_func=score_func, k=k).fit(self.X,self.y)
                    self.X = self.X.iloc[:,skb.get_support(indices=True)]

#           section to include optimization loop
            else:
                n = ''.join(k.split()).lower()
                n = n[:3]
                if n == 'opt':
                    pass              
        else:
            print('Please encode feature set first')
            
            
            
    def svd(self,n):
        if isinstance(n,int)==True or isinstance(n,float)==True:
            col = self.X.columns
            from sklearn.decomposition import TruncatedSVD
            svd = TruncatedSVD(n_components=n, n_iter=5, random_state=self.random_state).fit(self.X)
            self.X = pd.DataFrame(svd.transform(self.X), columns=['SV %i' % i for i in range(n)], index=self.X.index)  
#       section to include optimization loop
        else:
            n = ''.join(n.split()).lower()
            n = n[:3]
            if n == 'opt':
                pass             
            

            
    def pca(self,n):
        if isinstance(n,int)==True or isinstance(n,float)==True:
            from sklearn.decomposition import PCA
            pca = PCA(n_components=n, random_state=self.random_state).fit(self.X) 
            self.X = pd.DataFrame(pca.transform(self.X), columns=['PCA %i' % i for i in range(n)], index=self.X.index) 
#       section to include optimization loop
        else:
            n = ''.join(n.split()).lower()
            n = n[:3]
            if n == 'opt':
                pass             

    
#   def sample(strategy):
    def sample_tune_test(self,model,tuning_iter,tuning_strategy='randomized',tuning_metric='roc_auc',test_size=0.2):
        from sklearn.model_selection import train_test_split
        X_train, X_test, y_train, y_test = train_test_split(self.X, self.y, test_size=test_size, random_state=self.random_state)
#       function to automatically sample, HP tune, then test with best parameters
        col = X_train.columns
        from imblearn.under_sampling import RandomUnderSampler
        rus = RandomUnderSampler()
        X_rus, y_rus = rus.fit_sample(X_train, y_train)
        X_rus = pd.DataFrame(X_rus, columns = col)
        from imblearn.over_sampling import RandomOverSampler
        ros = RandomOverSampler()
        X_ros, y_ros = ros.fit_sample(X_train, y_train)
        X_ros = pd.DataFrame(X_ros, columns = col)
        from imblearn.over_sampling import SMOTE
        smote = SMOTE(ratio='minority')
        X_sm, y_sm = smote.fit_sample(X_train, y_train)
        X_sm = pd.DataFrame(X_sm, columns = col)
        from imblearn.combine import SMOTETomek
        smt = SMOTETomek(ratio='auto')
        X_smt, y_smt = smt.fit_sample(X_train, y_train)
        X_smt = pd.DataFrame(X_smt, columns = col)
        samples = [(X_train,y_train,"None"),
                  (X_rus,y_rus,"Random under sampling"),
                  (X_ros,y_ros,"Random over sampling"),
                  (X_sm,y_sm,"SMOTE"),
                  (X_smt,y_smt,"SMOTE + TL")] 
#       tune and test
        best_param=[0]*len(samples)
        self.metrics=pd.DataFrame(columns=["Sampling",
                                           "Accuracy",
                                           'Precision',
                                           'Recall',
                                           'Specificity',
                                           'Neg Pred Val',
                                           'Confusion Sum',
                                           'F1 score', 
                                           'F2 score',
                                           'Cohen kappa score',
#                                              "RMSE" : rmse,
#                                              "Jaccard score" : jaccard,
                                           "Brier score loss",
                                           'MCC',
                                           "AUC"])
        radar_df = pd.DataFrame(columns=['Sampling',
                                            'AUC',
                                            'F1', 
                                            'F2',
                                            'Cohen kappa',
                                            'Brier',
                                            'MCC'])
        model_rep=model
        print("Estimator: ",model_rep)
        from sklearn.metrics import cohen_kappa_score,accuracy_score,roc_auc_score,brier_score_loss,confusion_matrix,f1_score,recall_score,precision_score,matthews_corrcoef
        from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
        from skopt import BayesSearchCV
        from skopt.space import Real, Categorical, Integer
        from math import floor, ceil
        for i in range(0,len(samples)):
            model_inst=model_rep()
            print("Sampling technique: ",samples[i][2], "\n ")
#           IMPORTANT when inputting default hyperparameters:
#           wrap non int or float types with Categorical() function
#           NEVER input int or float types with only one parameter option... this will not work with bayes search (works fine with randomized search)
            if("RandomForest" in str(model_rep)):
                tree_ensemble=True
#               default hyperparameter testing range
                bootstrap = Categorical([True, False])
                n_estimators = [350,450]
                criterion= Categorical(['gini','entropy'])
                max_depth =np.arange(1,floor(len(X_train.columns)),1)
                max_features = np.random.uniform(0.01,1,10000)
                min_samples_split = np.random.uniform(0.01,1,10000)
                min_samples_leaf = np.random.uniform(0.0001,0.5,10000)
                class_weight = Categorical(['balanced','balanced_subsample',None])
                n_jobs = [-1]
                random_state = [self.random_state]
#               input hyperparameters into dictionary
                param_grid = dict(n_estimators=n_estimators,
                                  bootstrap=bootstrap, 
                                  criterion=criterion,
                                  min_samples_leaf=min_samples_leaf, 
                                  min_samples_split=min_samples_split,
                                  max_features=max_features,
                                  max_depth=max_depth,
                                  class_weight=class_weight,
                                  n_jobs=n_jobs,
                                  random_state=random_state)

            elif("XGB" in str(model_rep)):
                tree_ensemble=True
    #               default hyperparameter testing range
                booster = Categorical(['gbtree','gblinear','dart'])
                n_estimators = [350,450]
                learning_rate = np.random.uniform(0.000001,1,10000)
                max_depth = np.arange(1,floor(len(X_train.columns)),1)
                gamma = np.random.uniform(0,15,10000)
                reg_alpha = np.random.uniform(0,1,10000)
                reg_lambda = np.random.uniform(0,1,10000)
                objective = Categorical(['reg:logistic'])
                subsample = np.random.beta(2,5,10000)
                colsample_bytree = np.random.beta(2,5,10000)
                scale_pos_weight = np.random.uniform(0,20,10000)
                min_child_weight = np.random.uniform(0,0.5*floor(len(X_train.columns)),10000)
                max_delta_step = 0.2*floor(len(X_train.columns))*np.random.beta(2,5,10000)
                n_jobs = [-1]
                random_state = [self.random_state]
    #               input hyperparameters into dictionary
                param_grid = dict(n_estimators=n_estimators,
                                  booster=booster,
                                  learning_rate=learning_rate,
                                  max_depth=max_depth,
                                  gamma=gamma,
                                  reg_alpha=reg_alpha,
                                  reg_lambda=reg_lambda,
                                  objective=objective,
                                  subsample=subsample,
                                  colsample_bytree=colsample_bytree,
                                  scale_pos_weight=scale_pos_weight,
                                  min_child_weight=min_child_weight,
                                  max_delta_step=max_delta_step,
                                  n_jobs=n_jobs,
                                  random_state=random_state)          

            elif("LogisticRegression" in str(model_rep)):
                tree_ensemble=False
#               default hyperparameter testing range
                penalty = Categorical(['l2'])
                tol = np.random.uniform(0.00000001,0.01,10000)
                C = np.random.uniform(0.000001,1,10000)
                fit_intercept = Categorical([True,False])
                intercept_scaling = np.random.uniform(0.01,10,10000)
                class_weight = Categorical(['balanced',None])
                solver = Categorical(['newton-cg', 'lbfgs','sag'])
                max_iter = np.arange(100,1000,10)
                n_jobs = [-1]
                random_state = [self.random_state]
#               input hyperparameters into dictionary
                param_grid = dict(penalty=penalty,
                                  tol=tol,
                                  C=C,
                                  fit_intercept=fit_intercept,
                                  intercept_scaling=intercept_scaling,
                                  class_weight=class_weight,
                                  solver=solver,
                                  max_iter=max_iter,
                                  n_jobs=n_jobs,
                                  random_state=random_state)

            elif("Isolation" in str(model_rep)):
                tree_ensemble = True
#               default hyperparameter testing range
                bootstrap = Categorical([True, False])
                n_estimators = [350,450]
                max_samples= np.random.uniform(0.01,1,10000)
                max_features = np.random.uniform(0.01,1,10000)
                contamination = np.random.uniform(0,0.2,10000)
                behaviour = Categorical(['new'])
                n_jobs = [-1]
                random_state = [self.random_state]
#               input hyperparameters into dictionary
                param_grid = dict(n_estimators=n_estimators,
                                  bootstrap=bootstrap, 
                                  max_features=max_features,
                                  contamination=contamination,
                                  max_samples=max_samples,
                                  behaviour=behaviour,
                                  n_jobs=n_jobs,
                                  random_state=random_state)

            #elif("Bagging" in str(model)):

#           instantiate cv and grid
            kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=self.random_state)
            tuning_strategy=''.join(tuning_strategy.split()).lower()
            if tuning_strategy=='bayes':
                param_grid_cat = {k: v for k, v in param_grid.items() if isinstance(v,Categorical)==True or k=='n_jobs' or k=='random_state'}
                param_grid_num = {k:(min(v),max(v)) for k,v in param_grid.items() if k not in param_grid_cat}
                param_grid={**param_grid_num,**param_grid_cat}
                import warnings
                warnings.filterwarnings('ignore', message='The objective has been evaluated at this point before.')
                grid_search = BayesSearchCV(model_inst, param_grid, scoring=tuning_metric, n_jobs=-1, cv=kfold, n_iter=floor(tuning_iter/len(samples)),verbose=0)
            elif tuning_strategy=='randomized':
                grid_search = RandomizedSearchCV(model_inst, param_grid, scoring=tuning_metric, n_jobs=-1, cv=kfold, n_iter=floor(tuning_iter/len(samples)),verbose=0)
            print('Tuning...')
            grid_results=grid_search.fit(samples[i][0],samples[i][1])
            best_param[i]=grid_results.best_params_
            print("Best: {} using {} \n".format(grid_results.best_score_, best_param[i]))
            if(tree_ensemble==True):
                best_param[i].update({'n_estimators': 800})
            del grid_results
#           Train model
            model=model_rep
            model=model(**best_param[i])
            print("Fitting model...")
            model.fit(samples[i][0],samples[i][1])
            preds = model.predict(X_test[samples[i][0].columns])
            conf_mat = confusion_matrix(y_true=y_test, y_pred=preds)
            accuracy = accuracy_score(y_test, preds)
            ck = cohen_kappa_score(y_test,preds)
            brier = brier_score_loss(y_test,preds)
            auc = roc_auc_score(y_test, preds) 
            f1 = f1_score(y_test, preds)
#               jaccard = jaccard_score(y_test, preds)
            recall = recall_score(y_test, preds)
            precision = precision_score(y_test, preds)
            mcc = matthews_corrcoef(y_test, preds)
            specificity=round(conf_mat[0][0] / (conf_mat[0][0]+conf_mat[0][1]),5)
            neg_pred= round(conf_mat[0][0] / (conf_mat[0][0]+conf_mat[1][0]),5)
            f2=round(5*((precision*recall)/((4*precision)+recall)),5)
#               rmse = np.sqrt(mean_squared_error(y_test, preds))
            print("Accuracy:          ", accuracy)
            print('Precision:         ', precision)
            print('Recall:            ', recall)
            print('Specificity:       ', specificity)
            print('Neg Pred Val:      ', neg_pred)
            print('Confusion Sum:     ', precision+recall+specificity+neg_pred)                
            print(' ')
            print('F1 score:          ', f1)
            print('F2 score:          ', f2)
            print('Cohen kappa score: ', ck)
#               print("RMSE:              ", rmse)
            print(' ')
#               print("Jaccard score:     ", jaccard)
            print("Brier score loss:  ", brier)
            print('MCC:               ', mcc)
            print("AUC:               ", auc, "\n ","\n ","\n ","\n ","\n ") 
            df=pd.DataFrame({"Sampling" : samples[i][2],
                                           "Accuracy" : round(accuracy,5),
                                           'Precision' : round(precision,5),
                                           'Recall' : round(recall,5),
                                           'Specificity' : round(specificity,5),
                                           'Neg Pred Val' : round(neg_pred,5),
                                           'Confusion Sum' : round(precision+recall+specificity+neg_pred,5),
                                           'F1 score' : round(f1,5),
                                           'F2 score' : round(f2,5),
                                           'Cohen kappa score' : round(ck,5),
#                                              "RMSE" : rmse,
#                                              "Jaccard score" : jaccard,
                                           "Brier score loss" : round(brier,5),
                                           'MCC' : round(mcc,5),
                                           "AUC" : round(auc,5)},index=[i])
            df2 = pd.DataFrame({'Sampling': samples[i][2],
                                            'AUC': round(auc,5),
                                            'F1': round(f1,5),
                                            'F2': round(f2,5),
                                            'Cohen kappa' : round(ck,5),
                                            'Brier': round(brier,5),
                                            'MCC': round(mcc,5)},index=[i])
            self.metrics=self.metrics.append(df)
            radar_df=radar_df.append(df2)
        import matplotlib.pyplot as plt
        from math import pi
        # ------- RADAR CHARTS PART 1: Create background
        # number of variables
        categories=list(radar_df)[1:]
        N = len(categories)
        # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
        angles = [n / float(N) * 2 * pi for n in range(N)]
        angles += angles[:1]
        # Initialise the spider plot
        ax = plt.subplot(111, polar=True)
        # If you want the first axis to be on top:
        ax.set_theta_offset(pi / 2)
        ax.set_theta_direction(-1)
        # Draw one axe per variable + add labels labels yet
        plt.xticks(angles[:-1], categories)
        # Draw ylabels
        ax.set_rlabel_position(0)
        plt.yticks([0.2,0.4,0.6,0.8], ["0.2","0.4","0.6","0.8"], color="blue", size=7)
        plt.ylim(0,1)
        # ------- PART 2: Add plots
        # Plot each individual = each line of the data
        # Ind1
        values=radar_df.loc[2].drop('Sampling').values.flatten().tolist()
        values += values[:1]
        ax.plot(angles, values, linewidth=1, linestyle='solid', label="ROS")
        ax.fill(angles, values, 'b', alpha=0.1)
        # Ind2
        values=radar_df.loc[3].drop('Sampling').values.flatten().tolist()
        values += values[:1]
        ax.plot(angles, values, linewidth=1, linestyle='solid', label="SMOTE")
        ax.fill(angles, values, 'r', alpha=0.1)
        # Ind3
        values=radar_df.loc[1].drop('Sampling').values.flatten().tolist()
        values += values[:1]
        ax.plot(angles, values, linewidth=1, linestyle='solid', label="RUS")
        ax.fill(angles, values, 'k', alpha=0.1)
        # Ind4
        values=radar_df.loc[0].drop('Sampling').values.flatten().tolist()
        values += values[:1]
        ax.plot(angles, values, linewidth=1, linestyle='solid', label="No Sampling")
        ax.fill(angles, values, 'green', alpha=0.1)
        # Add legend
        plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
